<a href="https://colab.research.google.com/github/eduseiti/ia368v_dd_final/blob/master/split_clueweb22_pt_10M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys

In [2]:
IN_COLAB='google.colab' in sys.modules

In [3]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)

    PYTHON="python3"
else:
    WORKING_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/trabalho_final/"

    PỲTHON="python3.8"

Mounted at /content/drive


In [8]:
import tqdm
import glob
import json
import pandas as pd

### Some constants to control the corpus split

In [9]:
PASSAGES_PER_FILE=5e+6
CORPUS_PART_FILENAME_FORMAT="clueweb22-pt_colbertx_{:02}.tsv"

CORPUS_MAP_FILENAME = "clueweb22_pt_colbertx_to_clueweb_id.tsv"
CLUEWEB_COLBERT_CORPUS_PASSAGE_FORMAT="{}\t{}"

### From where to read the original passages and where to write the ColBERT-X converted data, already split in defined size

In [ ]:
ORIGINAL_CORPUS_FOLDER="/content/passage_to_bm25"
COLBERTX_CORPUS_FOLDER="/content"

### Convert and split the passages

In [10]:
original_corpus_files = glob.glob(os.path.join(ORIGINAL_CORPUS_FOLDER, "pt000*.json"))

In [11]:
len(original_corpus_files)

246

In [13]:
output_files_counter = 0
output_passages_counter = 0
output_file = open(os.path.join(COLBERTX_CORPUS_FOLDER, CORPUS_PART_FILENAME_FORMAT.format(output_files_counter)), "w")

colbertx_to_clueweb_id = []

tqdm_files = tqdm.tqdm(original_corpus_files, file=sys.stdout, position=0, leave=True)

for original_file in tqdm_files:

    tqdm_files.set_description(original_file)

    temp_passages = json.load(open(original_file))

    for passage in tqdm.tqdm(temp_passages, file=sys.stdout, position=0, leave=True):
        if output_passages_counter % PASSAGES_PER_FILE > 0:
            output_file.write("\n")

        colbertx_to_clueweb_id.append(passage['id'])

        output_file.write(CLUEWEB_COLBERT_CORPUS_PASSAGE_FORMAT.format(output_passages_counter,
                                                                       passage['content']))

        output_passages_counter += 1

        if output_passages_counter % PASSAGES_PER_FILE == 0:
            output_file.close()

            output_files_counter += 1

            output_file = open(os.path.join(COLBERTX_CORPUS_FOLDER, CORPUS_PART_FILENAME_FORMAT.format(output_files_counter)), "w")

output_file.close()

/content/passage_to_bm25/pt0000-22.json: 100%|██████████| 246/246 [07:38<00:00,  1.87s/it]


### Save the map to translate the ColBERT-X passages ID back to their original ID

In [14]:
with open(os.path.join(COLBERTX_CORPUS_FOLDER, CORPUS_MAP_FILENAME), "w") as output_file:
    for i, clueweb_id in enumerate(colbertx_to_clueweb_id):
        if i > 0:
            output_file.write("\n")

        output_file.write("{}\t{}".format(i, clueweb_id))

In [15]:
len(colbertx_to_clueweb_id)

20244544